In [6]:
from google.colab import drive
drive.mount('/content/drive')

root_dir = '/content/drive/MyDrive/cs224n/project'

%cd $root_dir

import sys
sys.path.insert(0, root_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/cs224n/project


In [ ]:
import json
import random
import html
import re
import bs4
import nltk
from collections import Counter
import matplotlib.pyplot as plt

import bashlint.bash as bash
from bashlint.data_tools import *

Setting bashlex grammar using file: /content/drive/MyDrive/cs224n/project/bashlint/grammar/grammar100.txt
Bashlint grammar set up (148 utilities)



In [ ]:
# read data
with open('nl2bash-data.json') as f:
    data = json.load(f)
    data = [x[1] for x in data.items()]
    
nls = [x['invocation'] for x in data]
cms = [x['cmd'] for x in data]

with open('nl2bash_nl.txt', 'w+') as f_nl:
    with open('nl2bash_cm.txt', 'w+') as f_cm:
        for nl, cm in zip(nls, cms):
            parsed = bash_parser(cm)
            try:
                template = ast2template(parsed)
            except:
                continue
            print(nl, file=f_nl)
            print(template, file=f_cm)



In [ ]:
# split train dev and test set
test_size = 200
dev_size = 200

with open('nl2bash_nl.txt') as f:
    nls = f.readlines()
with open('nl2bash_cm.txt') as f:
    cms = f.readlines()
    
pairs = list(zip(nls, cms))
random.shuffle(pairs)

test_pairs = pairs[:test_size]
pairs = pairs[test_size:]

val_pairs = pairs[:dev_size]
pairs = pairs[dev_size:]

train_pairs = pairs

def write_pairs(pairs, name):
    nls = [x[0] for x in pairs]
    cms = [x[1] for x in pairs]
    with open(name+'_nl.txt', 'w+') as f:
        f.write(''.join(nls))
    with open(name+'_cm.txt', 'w+') as f:
        f.write(''.join(cms))        
        
write_pairs(test_pairs, 'test')
write_pairs(val_pairs, 'dev')
write_pairs(train_pairs, 'train')

In [15]:
# further preprocess the data
EOS = '<|endoftext|>'
SEP1 = 'english:'
SEP2 = 'bash:'
def read_data(filename:str):
    with open(filename, 'r') as handle:
        content = handle.readlines()
    return [x.strip() for x in content]

def encode(prompt):
    return f"{EOS} {SEP1} {prompt}\n{SEP2}"

def context(pair):
    if '' in pair:
        return ''
    enc = f"{encode(pair[0])} {pair[1]}"
    return f"{enc} {EOS}\n"


def preprocess():
    for NAME in ('dev', 'train'):
        cm = read_data(NAME+'_cm.txt')
        nl = read_data(NAME+'_nl.txt')
        
        al = [context(x) for x in zip(nl, cm)]
        al = "".join(al)
        
        if not al.endswith(EOS):
            al += EOS
        with open(NAME + '.txt', 'w+') as handle:
            handle.write(al)

In [16]:
preprocess()

In [18]:
# a quick check
with open("train.txt", encoding="utf-8") as f:
    lines = [line for line in f.read().splitlines()]
    pairs = [x+"\n"+y for x,y in zip(lines[::2],lines[1::2])]
    print(pairs[0])
    print(pairs[1])

<|endoftext|> english: Print last day of April, 2009
bash: cal DateTime DateTime | tr Regex Regex | grep -v Regex | tail -n Quantity <|endoftext|>
<|endoftext|> english: Print file system disk space usage
bash: df <|endoftext|>
